In [ ]:
from datetime import datetime
import os

import aioinflux
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import numpy as np
import pandas as pd

In [ ]:
token_file = os.path.join(os.path.expanduser("~/"), ".influxdb")
with open(token_file, 'r') as fd:
    uname = fd.readline().strip()  # Can't hurt to be paranoid
    pwd = fd.readline().strip()

In [ ]:
client = aioinflux.InfluxDBClient(host='test-influxdb-efd.lsst.codes',
                                  port=443,
                                  ssl=True,
                                  username=uname,
                                  password=pwd,
                                  db='efd',
                                  output="dataframe")

In [ ]:
d1 = datetime(2019, 8, 23)
d2 = datetime(2019, 8, 24)

In [ ]:
query_base = "SELECT {0} FROM \"efd\".\"autogen\".\"lsst.sal.{1}\" WHERE time >= \'{2}\' AND time <= \'{3}\'"

In [ ]:
atcamera_df = await client.query(query_base.format('private_sndStamp,imageName,timeStamp',
                                                   'ATCamera.logevent_endReadout',
                                                   d1, d2))

In [ ]:
#atcamera_df

In [ ]:
atarchiver_df = await client.query(query_base.format('private_sndStamp,statusCode,description',
                                                     'ATArchiver.logevent_processingStatus',
                                                     d1, d2))

In [ ]:
#atarchiver_df

In [ ]:
aths_info = query_base.format('private_sndStamp,url',
                              'ATHeaderService.logevent_largeFileObjectAvailable',
                              d1, d2)
extra = ' AND \"generator\"=\'ATHeaderService\''
aths_info += extra

In [ ]:
aths_df = await client.query(aths_info)

In [ ]:
#aths_df

In [ ]:
atcamera_send_time = []
atarchiver_send_time = []
atarchiver_arrival_time = []
aths_send_time = []
#efd_send_time = []
for row in atcamera_df.itertuples():
    atcamera_time = row.private_sndStamp
    atcamera_image = row.imageName
    index = atarchiver_df['description'].str.contains(f"{atcamera_image}", na=False)
    try:
        if atarchiver_df['statusCode'][index].values[0] == 100.0:
            atarchiver_time = atarchiver_df['private_sndStamp'][index].values[0]
            have_archiver_file = True
        else:
            have_archiver_file = False
    except IndexError:
        have_archiver_file = False
    index = aths_df['url'].str.contains(f"{atcamera_image}", na=False)
    try:
        aths_time = aths_df['private_sndStamp'][index].values[0]
        have_header_file = True
    except IndexError:
        have_header_file = False
    #print(atarchive_send_time.values)
    #index = efd_df['url'].str.contains(f"{atcamera_image}", na=False)
    #try:
    #    efd_time = efd_df['private_sndStamp'][index].values[0]
    #    have_efd_header_file = True
    #except IndexError:
    #    have_efd_header_file = False
    
    if have_header_file and have_archiver_file:# and have_efd_header_file:
        atcamera_send_time.append(atcamera_time)
        atarchiver_send_time.append(atarchiver_time)
        aths_send_time.append(aths_time)
        #efd_send_time.append(efd_time)
        atarchiver_arrival_time.append(atarchiver_time - atcamera_time)

In [ ]:
atcamera_send_time_array = np.array(atcamera_send_time)
atarchiver_send_time_array = np.array(atarchiver_send_time)
atarchiver_arrival_time_array = np.array(atarchiver_arrival_time)
aths_send_time_array = np.array(aths_send_time)
#efd_send_time_array = np.array(efd_send_time)
time_diff_str = 'T^ATA_PS_Send - T^ATC_EOR_Send'

In [ ]:
hist = np.histogram(atarchiver_arrival_time_array, bins=np.arange(start=0, stop=100, step=2))
aat = hv.Histogram(hist, kdims=hv.Dimension('time differences', label=time_diff_str, unit='s'),
                   label='Distribution of 𝚫T')

In [ ]:
cst_vs_aat = hv.Scatter((atcamera_send_time_array.astype('datetime64[s]'), atarchiver_arrival_time_array),
                        kdims='T^ATC_EOR_Send', vdims=hv.Dimension('time difference', label=time_diff_str, unit='s'),
                        label='𝚫T During Image Taking')

In [ ]:
delta_t1 = (aths_send_time_array - atcamera_send_time_array)
min_x = np.min(delta_t1)
max_x = np.max(delta_t1)
hist = np.histogram(delta_t1, bins=np.linspace(min_x, max_x, 50))
ahst = hv.Histogram(hist, kdims=hv.Dimension('time differences', 
                                             label='T^ATHS_LFOA_Send - T^ATC_EOR_Send', unit='s'),
                    label='Distribution of 𝚫T1')

In [ ]:
#delta_t2 = (efd_send_time_array - aths_send_time_array)
#min_x = np.min(delta_t2)
#max_x = np.max(delta_t2)
#hist = np.histogram(delta_t2, bins=np.linspace(min_x, max_x, 50))
#est = hv.Histogram(hist, kdims=hv.Dimension('time differences', 
#                                             label='T^EFD_LFOA_Send - T^ATHS_LFOA_Send', unit='s'),
#                    label='Distribution of 𝚫T2')

In [ ]:
#delta_t3 = (efd_send_time_array - atcamera_send_time_array)
#min_x = np.min(delta_t3)
#max_x = np.max(delta_t3)
#hist = np.histogram(delta_t3, bins=np.linspace(min_x, max_x, 50))
#hfst = hv.Histogram(hist, kdims=hv.Dimension('time differences', 
#                                             label='T^EFD_LFOA_Send - T^ATC_EOR_Send', unit='s'),
#                    label='Distribution of 𝚫T3')

In [ ]:
#ahst + est + hfst
ahst

In [ ]:
%%opts Scatter [padding=(0.05, 0.05)]
#hv.renderer('bokeh').theme = None
aat + cst_vs_aat

In [ ]:
indexes = np.where(atarchiver_arrival_time_array > 40.0)
for c, a in zip(atarchiver_arrival_time_array[indexes], atcamera_send_time_array.astype('datetime64[s]')[indexes]):
    print(f"{c:.2f}\t{a}")